In [1]:
#Importing Gurobi and Numpy
from gurobipy import *
import numpy as np



In [2]:
#Defining all constants

#Percentshare array has all the departments share of percentage hours to be allocated
percentshare= np.array([48.4,4.2,25.3,7.4,5.3,9.5])

#target carries the actual number of hours to be allocated to each department
target=213.5*percentshare/100


#hours_matrix is a matrix with acutual number of hours in the week (rows- Rooms, Columns-days of week)
hours_matrix=np.matrix([[9,9,9 ,9 ,9], [9, 9, 9, 9, 8],[9, 9, 9, 9, 8],[9, 9, 9 ,9 ,8],[ 7.5, 7.5, 7.5, 7.5, 6.5]])

#dept- total departments, rooms-total rooms, days-total days of the week
dept=6
rooms=5
days=5

In [3]:
#Defining the model
assignment1a=Model()

#x is three dimensional array with (dept, rooms and days as dimensions in order)
x = assignment1a.addVars(dept,rooms,days,vtype=GRB.BINARY)
s=assignment1a.addVars(dept)

Academic license - for non-commercial use only


In [4]:
#underallocation_con is a constraint on underallocation calculation in percent of underallocation of hours
#penalty_con penalises only the rooms which are underallocated
#consistency_con makes sure each room at a particular time is allocated to not more than one dept
underallocation_con={}
penalty_con={}
consistency_con={}
for i in range(dept):
    available=0
    for j in range(rooms):
        
        for k in range(days):
            available=available+x[i,j,k]*hours_matrix[j,k]
    underallocation_con[i]=assignment1a.addConstr(s[i]>=(1-available/target[i]))
    penalty_con[i]=assignment1a.addConstr(s[i]>=0)

#fullallocation_con=assignment1a.addConstr(x.sum()==25)
for j in range(rooms):
    for k in range(days):
        consistency_con[j,k]=assignment1a.addConstr(sum(x[i,j,k] for i in range(dept))==1)




In [5]:
#objective function on S(our underalloction variable)
assignment1a.setObjective(sum((s[i]) for i in range(dept)),GRB.MINIMIZE)


In [6]:
assignment1a.update()

In [7]:
assignment1a.optimize()

Optimize a model with 37 rows, 156 columns and 312 nonzeros
Variable types: 6 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [6e-02, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.9760935
Presolve removed 6 rows and 0 columns
Presolve time: 0.00s
Presolved: 31 rows, 156 columns, 306 nonzeros
Variable types: 6 continuous, 150 integer (150 binary)

Root relaxation: objective 2.066116e-03, 56 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00207    0    8    0.97609    0.00207   100%     -    0s
H    0     0                       0.4228716    0.00207   100%     -    0s
H    0     0                       0.4211107    0.00207   100%     -    0s
     0     0    0.02836    0   10    0.42111    0.02836  93.3%     -    

In [564]:
#objective value of our model.. (only 0.05 % of hours are unallocated out of 213.5 hours)
opt_val = assignment1a.objval
print("Optimal underallocation of hours in percent:",opt_val)

x_opt = [x[i,j,k].x for i in range(dept) for j in range(rooms) for k in range(days)]
print(x_opt)

Optimal underallocation of hours in percent: 0.05190597648316659
[-0.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0, -0.0, 1.0, -0.0, 1.0, -0.0, -0.0, 1.0, 1.0, -0.0, 1.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 1.0, -0.0, -0.0, 1.0, -0.0, -0.0, -0.0, 1.0, -0.0, -0.0, -0.0, 1.0, -0.0, -0.0, -0.0, 1.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 1.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 1.0, 1.0, -0.0, 1.0, -0.0]


In [565]:
for i in range(dept):
    sum1=0
    for j in range(rooms):
        for k in range(days):
            sum1=sum1+(x[i,j,k].x)*hours_matrix[j,k]
    print("The hours & % underallocation in Dept",i,"is", sum1,"hours and", s[i].x,"% underallocation")

The hours & % underallocation in Dept 0 is 98.0 hours and 0.05161902181275907 % underallocation
The hours & % underallocation in Dept 1 is 9.0 hours and 0.0 % underallocation
The hours & % underallocation in Dept 2 is 54.0 hours and 0.00028695467040752015 % underallocation
The hours & % underallocation in Dept 3 is 16.0 hours and 0.0 % underallocation
The hours & % underallocation in Dept 4 is 14.0 hours and 0.0 % underallocation
The hours & % underallocation in Dept 5 is 22.5 hours and 0.0 % underallocation


In [566]:
#Defining the shifting constraint for (b) part
shifting=assignment1a.addVars(dept,days-1)  
for i in range(dept):
    for k in range(days-1):
        assignment1a.addConstr(shifting[i,k]<=x[i,0,k+1]-x[i,0,k])
        assignment1a.addConstr(shifting[i,k]>=x[i,0,k+1]-x[i,0,k])

assignment1a.addConstr(sum(shifting[i,k] for i in range(dept) for k in range(days-1))<=4)



<gurobi.Constr *Awaiting Model Update*>

In [567]:
assignment1a.update()
assignment1a


<gurobi.Model MIP instance Unnamed: 86 constrs, 180 vars, Parameter changes: LogFile=gurobi.log, CSIdleTimeout=1800>

In [568]:
assignment1a.optimize()

Optimize a model with 86 rows, 180 columns and 480 nonzeros
Variable types: 30 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [6e-02, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]

MIP start did not produce a new incumbent solution
MIP start violates constraint R41 by 1.000000000

Found heuristic solution: objective 1.3093312
Presolve removed 59 rows and 48 columns
Presolve time: 0.01s
Presolved: 27 rows, 132 columns, 258 nonzeros
Variable types: 6 continuous, 126 integer (126 binary)

Root relaxation: objective 2.066116e-03, 45 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00207    0   10    1.30933    0.00207   100%     -    0s
H    0     0                       0.5256595    0.00207   100%     -    0s
     0     0    0.02836    0   13    0.525

In [569]:
#objective value of our model.. (only 0.05 % of hours are unallocated out of 213.5 hours)
opt_val = assignment1a.objval
print("Optimal underallocation of hours in percent:",opt_val)

x_opt = [x[i,j,k].x for i in range(dept) for j in range(rooms) for k in range(days)]
print(x_opt)

Optimal underallocation of hours in percent: 0.05190597648318136
[1.0, 1.0, 1.0, 1.0, 1.0, -0.0, -0.0, -0.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0, 0.0, -0.0, -0.0, 1.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 1.0, 1.0, -0.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, 1.0, 1.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, -0.0, -0.0, -0.0, -0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 1.0, -0.0, 1.0, 1.0, -0.0]


In [570]:
for i in range(dept):
    sum1=0
    for j in range(rooms):
        for k in range(days):
            sum1=sum1+(x[i,j,k].x)*hours_matrix[j,k]
    print("The hours & % underallocation in Dept",i,"is", sum1,"hours and", s[i].x,"% underallocation")

The hours & % underallocation in Dept 0 is 98.0 hours and 0.05161902181276247 % underallocation
The hours & % underallocation in Dept 1 is 9.0 hours and 0.0 % underallocation
The hours & % underallocation in Dept 2 is 54.0 hours and 0.000286954670418893 % underallocation
The hours & % underallocation in Dept 3 is 16.0 hours and 0.0 % underallocation
The hours & % underallocation in Dept 4 is 14.0 hours and 0.0 % underallocation
The hours & % underallocation in Dept 5 is 22.5 hours and 0.0 % underallocation
